In [33]:
import pandas as pd
import numpy as np
import akshare as ak
import datetime

import tushare as ts
pro = ts.pro_api("823c5d4332aebab55e69af3d437b2db067cfb0fc5161382a2a691b6d")
pd.set_option('expand_frame_repr',False)
pd.set_option('display.max_rows',5000)

In [60]:
ts_code = '159902.SZ'
found_date = pro.fund_basic(ts_code=ts_code,fields='found_date').iloc[0,0]
found_date = pd.to_datetime(found_date,format='%Y%m%d')

df = pro.fund_nav(ts_code=ts_code)
df = df[['ts_code','nav_date','adj_nav']].sort_values(by='nav_date')
df = df.reset_index(drop=True)
df['nav_date'] = pd.to_datetime(df['nav_date'],format='%Y%m%d')
df['current_drawdown'] = 1 - df['adj_nav']/df['adj_nav'].cummax()
df['max_drawdown'] = df['current_drawdown'].cummax() 
df['回撤深度'] = df['current_drawdown']/df['max_drawdown']
df['上市时间'] = (df['nav_date']-found_date).dt.days

aa = 0.6
bb = 0.1
df[['signal','pos','最新资产','收益回撤']] = np.nan
k=0
df.loc[0,'最新资产'] = 1
for i in range(1,len(df)):
    if df['上市时间'].iloc[i] > 3650:
        if df['pos'].iloc[i-1] != 1:      # 上一期没有仓位
            df.loc[i,'最新资产'] = df.loc[i-1,'最新资产']
            if df['回撤深度'].iloc[i] > aa :
                df.loc[i,'signal'] = 1
                df.loc[i,'pos'] = 1
                k = i
        else:                                # 上一期有仓位
            df.loc[i,'最新资产'] = df.loc[i-1,'最新资产']*(1+df['adj_nav'].iloc[i]/df['adj_nav'].iloc[i-1]-1)
            df.loc[i,'最新资产'] = df.loc[i-1,'最新资产']*(df['adj_nav'].iloc[i]/df['adj_nav'].iloc[i-1])
            if df['signal'].iloc[i] != -1 :
                df.loc[i,'pos'] = 1
        df.loc[i,'收益回撤'] = 1 - df.loc[i,'最新资产']/df.iloc[k:i]['最新资产'].max()
        if df.loc[i,'收益回撤'] > bb and df.loc[i,'最新资产'] > df.loc[k,'最新资产']*1.3 and df['pos'].iloc[i-1] == 1:
            df.loc[i,'signal'] = -1
            df.loc[i,'pos'] = 0
    else:
        df.loc[i,'最新资产'] = df.loc[i-1,'最新资产']
                # if df['累计收益'].iloc[i] -1 < bb :
                #     df.loc[i,'signal'] = -1
                #     df.loc[i,'pos'] = 0


mask = df['signal'].notna() #| df['pos'].notna()
df[mask]


,ts_code,nav_date,adj_nav,current_drawdown,max_drawdown,回撤深度,上市时间,signal,pos,最新资产,收益回撤
2399,159902.SZ,2016-06-06,3.399700,0.417600,0.653649,0.638875,3651,1.0,1.0,1.000000,NaN
3563,159902.SZ,2021-02-26,5.014714,0.140934,0.653649,0.215611,5377,-1.0,0.0,1.475046,0.101071
4178,159902.SZ,2023-08-21,3.538072,0.393896,0.653649,0.602611,6283,1.0,1.0,1.475046,NaN
